In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agent-Projects")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agent-Projects


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

In [4]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

# 문서의 내용을 표시하는 템플릿을 정의합니다.
document_prompt = PromptTemplate.from_template(
    "<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>"
)

# retriever 를 도구로 정의합니다.
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search for information in the PDF file",
    document_prompt=document_prompt,
)

In [5]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.tools import tool

# DallE API Wrapper를 생성합니다.
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)


# DallE API Wrapper를 도구로 정의합니다.
@tool
def dalle_tool(query):
    """use this tool to generate image from text"""
    return dalle.run(query)

In [6]:
from langchain_community.agent_toolkits import FileManagementToolkit

# 작업 디렉토리 경로 설정
working_directory = "tmp"

# 파일 관리 도구 생성(파일 쓰기, 읽기, 디렉토리 목록 조회)
file_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["write_file", "read_file", "list_directory"],
).get_tools()

# 생성된 파일 관리 도구 출력
file_tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp')]

In [7]:
tools = file_tools + [
    retriever_tool,
    search,
    dalle_tool,
]

# 최종 도구 목록 출력
tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp'),
 Tool(name='pdf_search', description='use this tool to search for information in the PDF file', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000184B9554A40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000184BA10EB90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page', 'page_content', 'source'], input_types={}, partial_variables={}, template='<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000184B9803B00>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.v

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import AgentStreamParser

# session_id 를 저장할 딕셔너리 생성
store = {}

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "You are a professional researcher. "
            "You can use the pdf_search tool to search for information in the PDF file. "
            "You can find further information by using search tool. "
            "You can use image generation tool to generate image from text. "
            "Finally, you can use file management tool to save your research result into files.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


# LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

# 스트림 파서 생성
agent_stream_parser = AgentStreamParser()

In [9]:
# 에이전트 실행
result = agent_with_chat_history.stream(
    {
        "input": "삼성전자가 개발한 `생성형 AI` 와 관련된 유용한 정보들을 PDF 문서에서 찾아서 bullet point로 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 새롭게 생성하여 정리한 내용을 저장해주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 발췌한 PDF 문서의 페이지 번호, 파일명을 기입하세요(예시: page 10, filename.pdf). \n"
        "3. 정리된 bullet point를 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: <document><content>SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창

In [10]:
# 웹 검색을 통해 보고서 파일 업데이트
result = agent_with_chat_history.stream(
    {
        "input": "이번에는 삼성전자가 개발한 `생성형 AI` 와 관련된 정보들을 웹 검색하고, 검색한 결과를 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 열어서 기존의 내용을 읽고, 웹 검색하여 찾은 정보를 이전에 작성한 형식에 맞춰 뒷 부분에 추가해 주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 정보의 출처(url)를 기입하세요(예시: 출처: 네이버 지식백과). \n"
        "3. 정리된 웹검색 내용을 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: tavily_search_results_json
query: 삼성전자 생성형 AI
Log: 
Invoking: `tavily_search_results_json` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: [{'title': '[삼성전자 AI 리더십] ③ 사용자 경험을 최우선으로 하는 AI 전략', 'url': 'https://news.samsung.com/kr/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-ai-%EB%A6%AC%EB%8D%94%EC%8B%AD-%E2%91%A2-%EC%82%AC%EC%9A%A9%EC%9E%90-%EA%B2%BD%ED%97%98%EC%9D%84-%EC%B5%9C%EC%9A%B0%EC%84%A0%EC%9C%BC%EB%A1%9C-%ED%95%98%EB%8A%94', 'content': '개인의 일상을 넘어 기업에서도 생성형 AI의 활용도가 커지고 있다. 삼성전자는 AI 기술 경쟁력 확보, 민감한 사내 정보도 안전하게 사용할 수 있는 정보보호, 그리고 용도별 최적 규모와 성능의 모델 적용을 위해 자체 생성형 AI 모델을 개발했다. 이를 통해 다양한 제품과 사내·외 서비스에 최적화된 모델들을 확보하고, 더욱 안전한 환경에서 사내 생산성 향상을 지원하고 있다.\n\n최근 삼성전자는 자체 생성형 AI 모델 ‘삼성 가우스2(Samsung Gauss2)’를 공개했다. 삼성 가우스2는 지난해 처음 공개한 ‘삼성 가우스1’에서 성능과 효율성을 강화한 후속 모델이며, 여러 가지 유형의 데이터를 동시에 처리할 수 있는 멀티모달(Multimodal)[3] 모델로 확장됐다. [...] 클라우드 AI는 서버 상의 방대한 데이터와 대형 AI 모델을 기반으로 다양한 기능을 구현할 수 있다. 삼성전자는 고성능을 구현하는 동시에 효율과 보안 역시 만족할 수 있도록 데이터 지연(Latency) 최소화, 모델 효율화, 데이터 익명화·암호화 

In [11]:
# 보고서 작성을 요청합니다.
result = agent_with_chat_history.stream(
    {
        "input": "`report.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용을 바탕으로, 전문적인 수준의 보고서를 작성하세요. "
        "보고서는 총 3개의 섹션으로 구성되어야 합니다:\n"
        "1. 개요: 보고서 abstract 를 300자 내외로 작성하세요.\n"
        "2. 핵심내용: 보고서의 핵심 내용을 작성하세요. 정리된 표를 markdown 형식으로 작성하여 추가하세요. "
        "3. 최종결론: 보고서의 최종 결론을 작성하세요. 출처(파일명, url 등)을 표시하세요."
        "마지막으로 작성된 결과물을 `report-2.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: read_file
file_path: report.md
Log: 
Invoking: `read_file` with `{'file_path': 'report.md'}`



[관찰 내용]
Observation: ## 삼성전자 생성형 AI '삼성 가우스2' 관련 정보 (웹 검색)

### 출처
- 출처: [삼성전자 AI 리더십](https://news.samsung.com/kr/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-ai-%EB%A6%AC%EB%8D%94%EC%8B%AD-%E2%91%A2-%EC%82%AC%EC%9A%A9%EC%9E%90-%EA%B2%BD%ED%97%98%EC%9D%84-%EC%B5%9C%EC%9A%B0%EC%84%A0%EC%9C%BC%EB%A1%9C-%ED%95%98%EB%8A%94) 
- 출처: [지디넷코리아](https://zdnet.co.kr/view/?no=20241121083205) 
- 출처: [조선일보](https://www.chosun.com/economy/tech_it/2024/11/21/NPX26RN6MVDQTLBYTHWC4F2S44/) 

### 정리된 웹 검색 내용
- 삼성전자는 생성형 AI 모델 '삼성 가우스2'를 공개하였으며, 이는 여러가지 데이터 유형을 동시에 처리할 수 있는 멀티모달 모델이다.
- 삼성 가우스2는 언어, 코드, 이미지 통합으로 구성되며, 사용자에게 다양한 AI 기능을 제공하기 위해 제품에 적용될 예정이다.
- '삼성 가우스2'는 서비스 용도에 따라 콤팩트, 밸런스드, 슈프림 3가지 모델로 나뉘어져 있으며, 온디바이스 환경에서 최적화된 성능을 제공한다.
- 삼성전자는 사내에서의 소프트웨어 개발을 지원하기 위해 '코드아이'와 같은 AI 도구를 제공, 이를 통해 개발자들의 생산성을 높이고 있다.
- 삼성전자는 '모두를 위한 AI(AI for All)'라는 비전 하에 모든 제품군에 AI 서비스를 통합할 계획이며, 이를 통해 사용자 경험

In [12]:
# 이미지 생성을 요청합니다.
result = agent_with_chat_history.stream(
    {
        "input": "`report-2.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용에 어울리는 이미지를 생성하세요. "
        "생성한 이미지의 url 을 markdown 형식으로 보고서의 가장 상단에 추가하세요. "
        "마지막으로 작성된 결과물을 `report-3.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: read_file
file_path: report-2.md
Log: 
Invoking: `read_file` with `{'file_path': 'report-2.md'}`



[관찰 내용]
Observation: # 삼성전자 생성형 AI '삼성 가우스2' 보고서

## 개요  
본 보고서는 삼성전자가 개발한 생성형 AI 모델 '삼성 가우스2'에 대한 정보를 다룬다. 생성형 AI는 다양한 데이터 유형을 통합적으로 처리할 수 있는 멀티모달 모델로, 언어, 코드, 이미지를 포함한 다각적인 기능을 제공한다. 삼성전자는 이를 통해 사용자 경험을 향상시키고, 기업 내 소프트웨어 개발에 기여하고자 한다. 

## 핵심내용  
| 항목              | 설명                                                         |
|------------------|------------------------------------------------------------|
| 모델명           | 삼성 가우스2 (Samsung Gauss2)                              |
| 기능             | 언어, 코드, 이미지 통합 생성형 AI                          |
| 모델 유형        | 콤팩트, 밸런스드, 슈프림 3가지 모델로 나뉨                |
| 특징             | 온디바이스 환경에서 최적화된 성능 제공                     |
| 지원 도구        | 코드 개발 지원을 위한 AI 도구 '코드아이' 제공              |
| 비전             | '모두를 위한 AI(AI for All)'로 모든 제품군에 AI 통합 계획|

## 최종결론  
삼성전자가 개발한 생성형 AI '삼성 가우스2'는 산업 전반에 걸쳐 혁신적인 사용자 경험을 제공할 잠재력을 가지